# TP 5 Réseaux de neurones - Multi classification et Propagation en avant
Dans ce TP, nous aimerions faire une classification multiclasse. Pour ce faire, nous allons comparer entre une classification utilisant la regression logistique, et une classification en utilisant les réseaux de neurones.

L'ensemble de données que nous allons utiliser est le même que celui utilisé durant le TP1, i.e. les images de chiffres manuscrits.


# Importation des librairies necessaires au travail

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import random
import cv2 

# Lecture des fichiers de données
Pour ce TP, nous allons lire les données à partir d'un fichier csv.

In [120]:
# données
data = np.genfromtxt('data.csv', delimiter=',', dtype=float)
data.shape

(5000, 401)

Dans ces données (data), les 400 premières colonnes representent les pixels de l'image (20x20), la dernière colonne represente la classe de l'image (chiffres de 0 à 9). (http://yann.lecun.com/exdb/mnist/)

Chaque ligne represente un exemple de notre ensemble de données. 

Mettons ces données dans leus vecteurs correspondants.

In [244]:
# rajoutons l'ordonnée à l'origine theta 0
intercept=np.ones((data.shape[0],1))
X=np.column_stack((intercept,data[:,:-2]))
y=data[:,-1]
# forcer y à avoir une seule colonne
y = y.reshape( y.shape[0], 1)

In [240]:
print('X', X.shape ,' y ', y.shape)
print(data.shape)

X (5000, 400)  y  (5000, 1)
(5000, 401)


Visualisation aléatoire de quelques données

In [20]:
plt.figure(figsize=(15,8))
for i in range(13):
    c = random.randint(X.shape[0])
    a = X[c,:].reshape((20, 20))
    a=np.transpose(a)
    plt.subplot(1,13,i+1)
    plt.title('label '+ str(y[c]))
    plt.imshow(a,cmap='gray')

# Partie 1, régression logistique
Dans cette partie, nous aimerions utiliser la régression logistique pour classifier nos images

Rappelons que la régression logistique nous donne la probabilité d'appartenance (oui ou non) à la classe 1 (elle permet une classification binaire).

Pour étendre la régression logistique à une multi-classification, nous allons utiliser une stratégie 1 contre tous.

Nous paramètres $\theta$ seront donc une matrice avec un nombre de lignes égale au nombre de classes, et avec un nombre de colones égale au nombre decaractéristiques (chaque ligne de la matrice $\theta$ correspond aux paramètres d'un classifieur.

In [21]:
def Sigmoid(z):
    return 1/(1+np.exp(-z));

In [29]:
def lrCostFunction (X, y, theta, alpha, MaxIter,lambda_):
    for i in range(MaxIter):
        reduction = theta * lambda_/len(y)
        reduction[0][0] ##theta0 wont receive the reduction effect
        Trans_X = X.T
        y_hat = Sigmoid(np.matmul(X,theta))
        error = y_hat - y
        theta = theta - (alpha * (Trans_X.dot(error) + reduction))
    return theta.T

In [160]:
def maximum_indexes(y_hat):
    y_pred = np.zeros((y_hat.shape[0],1))
    for i in range(y_hat.shape[0]):
        max = 0
        for j in range(y_hat.shape[1]):
            if(y_hat[i,j] > y_hat[i,max]):
                max = j
        y_pred[i,0] = max
    return y_pred

In [161]:
def predictOneVsAll(all_theta, X):
    y_hat = Sigmoid(X @ all_theta.T)
    y_pred = maximum_indexes(y_hat)
    return y_pred

In [18]:
classes= np.unique(y)
number_classes=classes.shape[0]
all_theta = np.zeros((number_classes, X.shape[1]));
all_theta.shape

(10, 400)

In [39]:
MaxIter= 10000
lambda_= 0.1
alpha = 0.01
# initial_theta pour chaque classifieur
initial_theta=np.zeros((X.shape[1], 1));
for i in range (number_classes):
     theta = lrCostFunction(X,(y==classes[i]).astype(int),initial_theta,alpha, MaxIter,lambda_);
     all_theta[i,:]=theta;                      

/tmp/ipykernel_6284/3487258107.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-z));


In [252]:
number_classes

10

In [242]:
y_pred = predictOneVsAll(all_theta, X);
print(y_pred)


[[0.]
 [0.]
 [0.]
 ...
 [9.]
 [9.]
 [7.]]


# Qualité du classifieur RL

Prédire des valeurs de y

Ici il serait interessant de calculer la précision de notre classifieur

Essayons de calculer ça avec 

moyenne(y==y-pred) * 100

Ceci donnera un pourcentage de precision

In [243]:
precision = np.mean(y==y_pred)*100
precision


95.26

# Partie 2: Réseaux de neurones
Pour cette partie, nous choisissons une réseau simple:

- une couche d'entrée avec 400 noeuds (20 x 20 pixels) + le biais
- une couche cachée avec 25 noeuds
- une couche de sortie avec 10 noeuds (nombre de classes)

In [174]:
# poids de la couche 1
W1 = np.genfromtxt('W1.csv', delimiter=',', dtype=float)
W1.shape


(25, 401)

In [175]:
# poids de la couche 2
W2 = np.genfromtxt('W2.csv', delimiter=',', dtype=float)
W2.shape

(10, 26)

In [106]:
input_layer_size  = 400;  
hidden_layer_size = 25;   
num_labels = 10;

**Prédiction**

Appliquer une propagation en avant en utilisant les paramètres données pour prédir les classes de l'ensemble d'apprentissage.



In [187]:
def predict (W1, W2, X):
    output1 = Sigmoid(X @ W1.T)
    intercept=np.ones((data.shape[0],1))
    output1 = np.column_stack((intercept,output1))
    output2 = Sigmoid(output1 @ W2.T)
    print(output2.shape)
    y_pred = maximum_indexes(output2)
    y_pred = y_pred + 1
    y_pred[y_pred==10] = 0
    return y_pred

In [191]:
intercept=np.ones((data.shape[0],1))
X=np.column_stack((intercept,data[:,:-1]))
y_pred_nl = predict(W1, W2, X)
precision = np.mean(y==y_pred_nl)*100
precision

(5000, 10)


97.52

# Vérification de l'implementation
Comparer vos algorithmes à ceux de scikitlearn

In [192]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import random

def compareTO_sklearn(): 
    comp = LogisticRegression(max_iter=10000)
    comp.fit(data[:,:-2], data[:,-1])
    y_prob = comp.predict_proba(data[:,:-2])
    y_pred_sk = maximum_indexes(y_prob)

    print("Sklearn succes rate:",np.mean(y==y_pred_sk)*100)
    print("Our Linear regression success rate :",np.mean(y==y_pred)*100)
    print("Our neural network succes rate:",np.mean(y==y_pred_nl)*100)


In [235]:
compareTO_sklearn()

Sklearn succes rate: 96.24000000000001
Our Linear regression success rate : 95.26
Our neural network succes rate: 97.52


# Renforcement d'apprentissage
Mettre ici toute idée qui pourrait renforcer votre apprentissage

In [249]:
##Decrease alpha

X=np.column_stack((intercept,data[:,:-2]))
MaxIter= 3000
lambda_= 0.1
alpha = 0.001
# initial_theta pour chaque classifieur
initial_theta=np.zeros((X.shape[1], 1));
all_theta_im = np.zeros((number_classes, X.shape[1]));

theta = lrCostFunction(X,(y==classes[i]).astype(int),initial_theta,alpha, MaxIter,lambda_); 
for i in range (number_classes):
     theta = lrCostFunction(X,(y==classes[i]).astype(int),initial_theta,alpha, MaxIter,lambda_);
     all_theta_im[i,:]=theta;                

y_pred = predictOneVsAll(all_theta_im, X);
print(y_pred)

precision = np.mean(y==y_pred)*100
precision



[[0.]
 [0.]
 [0.]
 ...
 [9.]
 [9.]
 [7.]]


95.32000000000001

# Consignes

Le travail est à remettre par groupe de 4 au maximum [1..4].

Le délai est le vendredi 22 Avril 2022 à 22h


In [ ]:
# bonne chance